In [ ]:
#Tarig's part
import pandas as pd

In [62]:
df_sensors = pd.read_csv('air_quality_sensors_data.csv')
df_stations = pd.read_csv('air_quality_stations.csv')

#df_sensors.rename(columns={'idSensore': 'sensor_id', 'Data': 'date&time', 'Valore':'Value', 'Stato':'State', 'idOperatore':'operator_id'}, inplace=True)

sensor_list = df_sensors['idSensore'].unique()

df_common_sensors = df_stations[df_stations['IdSensore'].isin(sensor_list)]

df_common_sensors.to_csv('common_sensors.csv', index=False)


In [ ]:
common_sensors = pd.read_csv('common_sensors.csv')

df_sensors_data = pd.read_csv('air_quality_sensors_data.csv')

# For sensors where DataStop is blank, we will set them to 'Present' inside CSV file
# For sensors where DataStop is not 'Present', we will remove them from the dataset
common_sensors = common_sensors[common_sensors['DataStop']=='Present']

common_sensors.to_csv('common_sensors_present.csv', index=False)

common_sensor_list = common_sensors['IdSensore'].unique()

df_sensors_data = df_sensors_data[df_sensors_data['idSensore'].isin(common_sensor_list)]

In [ ]:

df_sensors_data['Data'] = pd.to_datetime(df_sensors_data['Data'], format='%d-%m-%Y %H:%M:%S')

cutoff_date = pd.to_datetime('01-01-2022 00:00:00', format='%d-%m-%Y %H:%M:%S')

df_sensors_data = df_sensors_data[df_sensors_data['Data'] > cutoff_date]

df_sensors_data.to_csv('present_sensors_data_01_Jan_2022.csv', index=False)

In [ ]:
df_stations_data = pd.read_csv('common_sensors.csv')
df_sensors_data = pd.read_csv('present_sensors_data_01_Jan_2022.csv')

df_sensors_data_list = df_sensors_data['idSensore'].unique()

df_stations_data = df_stations_data[df_stations_data['IdSensore'].isin(df_sensors_data_list)]

df_stations_data.to_csv('common_sensors_stations.csv', index=False)



In [ ]:
#Viplove's Part
df_common_sensors = pd.read_csv('common_sensors_stations.csv')
df_common_sensors_data = pd.read_csv('present_sensors_data_01_Jan_2022.csv')

#df_common_sensors_data = df_common_sensors_data.merge(df_common_sensors, left_on='idSensore', right_on='IdSensore', how='left')

df_common_sensors.drop(columns=['Quota','Comune','Storico','Utm_Nord','UTM_Est','Location'], inplace=True)

df_common_sensors_data.drop(columns=['Stato','idOperatore'], inplace=True)

df_common_sensors_data = df_common_sensors_data.rename(columns={
    'idSensore': 'sensor_id',
    'Data': 'date_time',
    'Valore': 'value'})

df_common_sensors = df_common_sensors.rename(columns={
    'IdSensore': 'sensor_id',
    'NomeTipoSensore': 'sensor_type',
    'UnitaMisura': 'unit',
    'Idstazione': 'station_id',
    'NomeStazione': 'station_name',
    'Provincia': 'province',
    'DataStart': 'start_date',
    'DataStop': 'end_date','lng': 'longitude',
    'lat': 'latitude'})

df_common_sensors.to_csv('sensors_and_station_info.csv', index=False)
df_common_sensors_data.to_csv('sensors_data_01_Jan_2022.csv', index=False)

In [59]:
df_sensors_data_01_Jan_2022 = pd.read_csv('sensors_data_01_Jan_2022.csv')
df_sensors_and_station_info = pd.read_csv('sensors_and_station_info.csv')
df_sensors_data_01_Jan_2022 = df_sensors_data_01_Jan_2022.merge(df_sensors_and_station_info, on='sensor_id', how='left')

df_sensors_data_01_Jan_2022.to_csv('sensors_data_01_Jan_2022_to_present_with_station_info.csv', index=False)

df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('PM10 (SM2005)', 'PM10')
df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('Particelle sospese PM2.5', 'PM2.5')
df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('Biossido di Azoto', 'Nitrogen Dioxide(NO2)')
df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('Ozono', 'Ozone(O3)')
df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('Ossidi di Azoto', 'Nitrogen Oxides(NOx)')
df_sensors_data_01_Jan_2022['sensor_type'] = df_sensors_data_01_Jan_2022['sensor_type'].str.replace('Monossido di Carbonio', 'Carbon Monoxide(CO)')

df_sensors_data_01_Jan_2022 = df_sensors_data_01_Jan_2022[df_sensors_data_01_Jan_2022['sensor_type']!='Benzene']

df_sensors_data_01_Jan_2022.to_csv('sensors_data_01_Jan_2022_to_present_with_station_info_english.csv', index=False)

In [ ]:
df_sensors_data_01_Jan_2022 = pd.read_csv('sensors_data_01_Jan_2022_to_present_with_station_info_english.csv')

df_sensors_data_01_Jan_2022 = df_sensors_data_01_Jan_2022[df_sensors_data_01_Jan_2022['value']!=-9999]

df_sensors_data_01_Jan_2022.to_csv('sensors_data_lombardia_final.csv', index=False)




In [12]:
df_sensors_data = pd.read_csv('sensors_data_lombardia_final.csv')

df_sensors_data['date_time'] = pd.to_datetime(df_sensors_data['date_time'], format='%d-%m-%Y %H:%M')

df_sorted = df_sensors_data.sort_values(by=['sensor_id', 'date_time'])

df_sensor_station_info = df_sorted.drop_duplicates(subset='sensor_id', keep='last')

df_sensor_station_info = df_sensor_station_info[['sensor_id', 'date_time', 'value', 'sensor_type', 'latitude', 'longitude']]


In [13]:
def classify_air_quality(row):
    pollutant = row['sensor_type']
    value = row['value']

    if 'NO2' in pollutant or 'NOx' in pollutant:
        if value <= 100:
            return 'Good'
        elif value <= 200:
            return 'Moderate'
        else:
            return 'Unhealthy'

    elif 'CO' in pollutant:
        if value <= 10:
            return 'Good'
        elif value <= 30:
            return 'Moderate'
        else:
            return 'Unhealthy'

    elif 'O3' in pollutant:
        if value <= 100:
            return 'Good'
        elif value <= 180:
            return 'Moderate'
        else:
            return 'Unhealthy'

    elif 'PM2.5' in pollutant:
        if value <= 35:
            return 'Good'
        elif value <= 100:
            return 'Moderate'
        else:
            return 'Unhealthy'

    elif 'PM10' in pollutant:
        if value <= 50:
            return 'Good'
        elif value <= 150:
            return 'Moderate'
        else:
            return 'Unhealthy'

    return 'Unknown'

# Apply classification
df_sensor_station_info['air_quality'] = df_sensor_station_info.apply(classify_air_quality, axis=1)

df_sensor_station_info.to_csv('distinct_sensor_info_lombardia_final.csv')
